In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
import networkx as nx
import pandas as pd
from collections import defaultdict
sys.path.insert(0, Path.cwd().absolute())

BASE_DIR = (Path.cwd()).parent

In [2]:
from network_analysis.flow_assignment import FlowAssignment
from geoprocessing.faf_zone_network import FafZoneNetwork, link_weights
from network_analysis.mc_flow_assignment import sample_flows
import geopandas as gpd



In [ ]:
naRailNodesDf = gpd.read_file( BASE_DIR / "inputs/NTAD_North_American_Rail_Network_Nodes")
naRailLinesDf = gpd.read_file( BASE_DIR / "inputs/NTAD_North_American_Rail_Network_Lines")
fafFlowDf = pd.read_csv(BASE_DIR / 'inputs/faf_freight_flow.csv' )
fafZonesDf = gpd.read_file( BASE_DIR / "inputs/2017_CFS_Metro_Areas_with_FAF").to_crs(4326)
fafZonesDf.FAF_Zone = fafZonesDf.FAF_Zone.astype(int)


In [4]:
fafZonesDf.loc[fafZonesDf.geometry.geom_type == 'MultiPolygon'].geom_type.value_counts()

MultiPolygon    21
Name: count, dtype: int64

In [5]:

fafZonesDfExploded = fafZonesDf.explode()

explodedZones = fafZonesDfExploded.loc[fafZonesDfExploded.FAF_Zone.duplicated()].FAF_Zone.unique()

explodedZoneRefDf = fafZonesDf.loc[fafZonesDf.FAF_Zone.isin(explodedZones)].copy()
explodedZoneRefDf['total_area'] = explodedZoneRefDf.area

fafZonesDfExploded = fafZonesDfExploded.join(
  other=explodedZoneRefDf.set_index(explodedZoneRefDf.FAF_Zone)[['total_area']],
  on=fafZonesDfExploded.FAF_Zone,
  how='left'
)
fafZonesDfExploded['area'] = fafZonesDfExploded.area
fafZonesDfExploded['area_frac'] = fafZonesDfExploded.area / fafZonesDfExploded.total_area
fafZonesDfExploded['area_frac'] = fafZonesDfExploded.area_frac.fillna(1)


SPLIT_IF_ABOVE_AREA_FRAC = .25
zonesWorthExplodingDf = fafZonesDfExploded.loc[
  fafZonesDfExploded.FAF_Zone.isin(explodedZones)
  & (fafZonesDfExploded.area_frac > SPLIT_IF_ABOVE_AREA_FRAC)
]
explodedZoneDf = zonesWorthExplodingDf.loc[zonesWorthExplodingDf.FAF_Zone.duplicated()]
zonesWorthExploding = explodedZoneDf.FAF_Zone.unique()
# fafZonesDfExploded = zonesWorthExplodingDf

fafZonesDfExploded['FAF_Zone_Real'] = fafZonesDfExploded.FAF_Zone

newFAFZones = []
seenFAFZones = defaultdict(int)
for i,r in fafZonesDfExploded.iterrows():
  if r.FAF_Zone_Real in zonesWorthExploding:
    seenFAFZones[r.FAF_Zone_Real] += 1
  newFAFZones.append(r.FAF_Zone_Real + seenFAFZones[r.FAF_Zone_Real] * 1000)
fafZonesDfExploded.FAF_Zone = newFAFZones


/tmp/ipykernel_29894/492442948.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  explodedZoneRefDf['total_area'] = explodedZoneRefDf.area
/tmp/ipykernel_29894/492442948.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fafZonesDfExploded['area'] = fafZonesDfExploded.area
/tmp/ipykernel_29894/492442948.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fafZonesDfExploded['area_frac'] = fafZonesDfExploded.area / fafZonesDfExploded.total_area


In [6]:
fafZonesDfExploded.to_file(BASE_DIR / 'inputs/faf_zones_exploded/faf_zones_exploded.shp')

/tmp/ipykernel_29894/2272871461.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fafZonesDfExploded.to_file(BASE_DIR / 'inputs/faf_zones_exploded/faf_zones_exploded.shp')
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'FAF_Zone_Real' to 'FAF_Zone_R'
  ogr_write(
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 2097880370 of field ALAND of feature 0 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 475262778 of field ALAND of feature 1 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
/home/nathanczh/miniconda3/envs/networ

In [7]:

def explodeFafFlows(fafFlowDf, zoneRatioDf, field):
  print("Joining")
  explodedFafFlowDf = fafFlowDf.join(
    other=zoneRatioDf.set_index('FAF_Zone_Real'),
    on=field,
    how='left'
  ).reset_index(drop=True)

  print("Joined")
  # explodedFafFlowDf.FAF_Zone
  flowColumns = [ c for c in explodedFafFlowDf.columns if c.startswith('thousand tons')]
  subAreas = explodedFafFlowDf[field] != explodedFafFlowDf.FAF_Zone
  areasToDivide = subAreas, flowColumns
  print('multiplying', explodedFafFlowDf.loc[areasToDivide].shape, explodedFafFlowDf.loc[subAreas, 'area_frac'].shape)
  explodedFafFlowDf.loc[areasToDivide] = explodedFafFlowDf.loc[areasToDivide].mul(explodedFafFlowDf.loc[subAreas, 'area_frac'], axis=0)
  explodedFafFlowDf[field] = explodedFafFlowDf.FAF_Zone
  explodedFafFlowDf = explodedFafFlowDf.drop(columns=['FAF_Zone', 'area_frac'])
  return explodedFafFlowDf


zone_ratios = fafZonesDfExploded[['FAF_Zone', 'FAF_Zone_Real', 'area_frac']]

explodedFafFlowDf = explodeFafFlows(fafFlowDf, zone_ratios, 'dms_orig')
display(explodedFafFlowDf.shape)
explodedFafFlowDf = explodeFafFlows(explodedFafFlowDf.copy(), zone_ratios.copy(), 'dms_dest')
display(explodedFafFlowDf.shape)
explodedFafFlowDf.to_csv(BASE_DIR / 'inputs/faf_freight_flow_exploded.csv', index=False)

Joining
Joined
multiplying (21865, 25) (21865,)


(175312, 29)

Joining
Joined
multiplying (33171, 25) (33171,)


(305433, 29)

In [8]:
# explodedFafFlowDf.loc[areasToDivide] * explodedFafFlowDf.loc[subAreas, 'area_frac']
explodedFafFlowDf[explodedFafFlowDf.index.duplicated()]


,dms_orig,dms_dest,sctg2,dms_mode,thousand tons in 2017,thousand tons in 2018,thousand tons in 2019,thousand tons in 2020,thousand tons in 2021,thousand tons in 2022,...,thousand tons in 2035_high,thousand tons in 2040,thousand tons in 2040_low,thousand tons in 2040_high,thousand tons in 2045,thousand tons in 2045_low,thousand tons in 2045_high,thousand tons in 2050,thousand tons in 2050_low,thousand tons in 2050_high


In [ ]:
fzn = FafZoneNetwork(naRailNodesDf, naRailLinesDf, fafZonesDfExploded).setInputDfColType()
faf_dir = BASE_DIR / 'resources/shapefiles/faf_network_exploded'
faf_dir.mkdir(parents=True, exist_ok=True)
fzn.fafZoneLinksDf[[c for c in fzn.fafZoneLinksDf.columns if not (c.endswith('_to') or c.endswith('_fr')) ]].to_file(faf_dir / 'faf_links.shp')
fzn.fafZoneNodesDf.to_file(faf_dir / 'faf_nodes.shp')

/tmp/ipykernel_29894/1861710640.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fzn.fafZoneLinksDf[[c for c in fzn.fafZoneLinksDf.columns if not (c.endswith('_to') or c.endswith('_fr')) ]].to_file(faf_dir / 'faf_links.shp')
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'FAF_Zone_fr' to 'FAF_Zone_f'
  ogr_write(
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'FAF_Zone_to' to 'FAF_Zone_t'
  ogr_write(
/tmp/ipykernel_29894/1861710640.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fzn.fafZoneNodesDf.to_file(faf_dir / 'faf_nodes.shp')
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normali

In [10]:
import folium
import branca.colormap as cm
from IPython.display import display

min_lat=24.7433195
max_lat=49.3457868
min_lon=-124.7844079
max_lon=-66.9513812


m = folium.Map(
  max_bounds=True,
  location=[42, -95],
  zoom_start=4,
  min_zoom=4,
  tiles='OpenStreetMap',
  max_lat=max_lat,
  min_lon=min_lon,
  max_lon=max_lon,
  min_lat=min_lat
)

zoneDf = fzn.fafZonesDf.copy()
zoneDf.geometry = zoneDf.geometry.simplify(.15)
zoneDf[['FAF_Zone_1', 'geometry']].explore(
  m=m,
  column='FAF_Zone_1',
  cmap='Greens',
  legend=False,
  style_kwds={'opacity':.1}
)

fzn.fafZoneLinksDf[['FAF_Zone_1_fr', 'FAF_Zone_1_to', 'geometry']].explore(
  m=m,
  style_kwds={
    'style_function': lambda feature: {
      'color': 'black',
      'weight': 1
    }
  }
)

fzn.fafZoneNodesDf[['FAF_Zone_1', 'geometry']].explore(
  m=m,
  color='black'
)

folium.LayerControl().add_to(m)

display(m)

In [14]:
fafZonesDf = gpd.read_file( BASE_DIR / "inputs/2017_CFS_Metro_Areas_with_FAF").to_crs(4326)
fafZonesDf.FAF_Zone = fafZonesDf.FAF_Zone.astype(int)

fzn = FafZoneNetwork(naRailNodesDf, naRailLinesDf, fafZonesDf).setInputDfColType()
faf_dir = BASE_DIR / 'resources/shapefiles/faf_network'
faf_dir.mkdir(parents=True, exist_ok=True)
fzn.fafZoneLinksDf[[c for c in fzn.fafZoneLinksDf.columns if not (c.endswith('_to') or c.endswith('_fr')) ]].to_file(faf_dir / 'faf_links.shp')
fzn.fafZoneNodesDf.to_file(faf_dir / 'faf_nodes.shp')

/tmp/ipykernel_29894/777385116.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fzn.fafZoneLinksDf[[c for c in fzn.fafZoneLinksDf.columns if not (c.endswith('_to') or c.endswith('_fr')) ]].to_file(faf_dir / 'faf_links.shp')
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'FAF_Zone_fr' to 'FAF_Zone_f'
  ogr_write(
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'FAF_Zone_to' to 'FAF_Zone_t'
  ogr_write(
/tmp/ipykernel_29894/777385116.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fzn.fafZoneNodesDf.to_file(faf_dir / 'faf_nodes.shp')
/home/nathanczh/miniconda3/envs/network_flow_assignment/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalize